<a href="https://colab.research.google.com/github/doukansurel/Retrieval-Augmented-Generation/blob/main/RetryQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerekli Kütüphanelerin Kurulumu ve Aktarılması

In [13]:
!pip install llama-index pypdf

ERROR: Operation cancelled by user


In [14]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from llama_index.indices.vector_store.base import VectorStoreIndex
from llama_index.readers.file.base import SimpleDirectoryReader

# Needed for running async functions in Jupyter Notebook
import nest_asyncio

nest_asyncio.apply()

In [16]:
import os

os.environ["OPENAI_API_KEY"] ="sk-OPENAI_API_KEY"

#Belgelerin Yüklenmesi

In [17]:
documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/RAG/Belgeler/satis-giderleri-tarifesi-4-1.pdf"]).load_data()
index = VectorStoreIndex.from_documents(documents)
query = "Sicile kayıtlı motorlu kara araçlarına ilişkin satış giderleri ile taşınmaz mallara, taşınır mallara ve diğer mallara ilişkin satış giderleri arasındaki farklılıkları açıklar mısınız?"

In [18]:
base_query_engine = index.as_query_engine()
response = base_query_engine.query(query)
print(response)

Sicile kayıtlı motorlu kara araçlarına ilişkin satış giderleri, aracın muhafazası, kıymet takdiri ve satış giderleri esas alınarak belirlenirken, taşınmaz mallara ilişkin satış giderleri, sadece kıymet takdiri ve satış giderleri esas alınarak belirlenir. Taşınır mallara ilişkin satış giderleri ise yine sadece kıymet takdiri ve satış giderleri esas alınarak belirlenir. Diğer mallara ilişkin satış giderleri ise niteliğine uygun düştüğü ölçüde satış gideri tablosunda yer alan ücretler esas alınır.


# Yeniden Deneme Sorgu Motoru
Temel sorgu motorundan gelen yanıtı iyileştirmek için bir değerlendirici kullanır.

Aşağıdakileri yapar:

önce temel sorgu motorunu sorgular, ardından
Yanıtın geçip geçmediğine karar vermek için değerlendiriciyi kullanın.
Yanıt geçerse yanıtı döndürün,
Aksi takdirde, orijinal sorguyu değerlendirme sonucuyla (sorgu, yanıt ve geri bildirim) yeni bir sorguya dönüştürün,
Maksimum_yeniden denemeye kadar tekrarlayın

In [19]:
from llama_index.query_engine import RetryQueryEngine
from llama_index.evaluation import RelevancyEvaluator

query_response_evaluator = RelevancyEvaluator()
retry_query_engine = RetryQueryEngine(
    base_query_engine, query_response_evaluator
)
retry_response = retry_query_engine.query(query)
print(retry_response)

Sicile kayıtlı motorlu kara araçlarına ilişkin satış giderleri, aracın muhafazası, kıymet takdiri ve satış giderleri esas alınarak belirlenirken, taşınmaz mallara ilişkin satış giderleri, sadece kıymet takdiri ve satış giderleri esas alınarak belirlenir. Taşınır mallara ilişkin satış giderleri ise yine sadece kıymet takdiri ve satış giderleri esas alınarak belirlenir. Diğer mallara ilişkin satış giderleri ise niteliğine uygun düştüğü ölçüde satış gideri tablosunda yer alan ücretler esas alınır.


# Kaynak Sorgu Motorunu Yeniden Deneme
Kaynağı Yeniden Deneme, sorgu için mevcut kaynak düğümlerini llm düğüm değerlendirmesine göre filtreleyerek sorgu kaynak düğümlerini değiştirir.

In [20]:
from llama_index.query_engine import RetrySourceQueryEngine

retry_source_query_engine = RetrySourceQueryEngine(
    base_query_engine, query_response_evaluator
)
retry_source_response = retry_source_query_engine.query(query)
print(retry_source_response)

Sicile kayıtlı motorlu kara araçlarına ilişkin satış giderleri, aracın muhafazası, kıymet takdiri ve satış giderleri esas alınarak belirlenirken, taşınmaz mallara ilişkin satış giderleri, kıymet takdiri ve satış giderleri esas alınarak belirlenir. Taşınır mallara ilişkin satış giderleri ise yine kıymet takdiri ve satış giderleri esas alınarak belirlenir. Diğer mallara ilişkin satış giderleri ise niteliğine uygun düştüğü ölçüde satış gideri tablosunda yer alan ücretler esas alınır.


# Yönerge Sorgu Motorunu Yeniden Deneme
Bu modül, değerlendiricinin davranışını yönlendirmek için yönergeleri kullanmaya çalışır.

In [21]:
from llama_index.evaluation.guideline import (
    GuidelineEvaluator,
    DEFAULT_GUIDELINES,
)
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)
from llama_index.query_engine.retry_query_engine import (
    RetryGuidelineQueryEngine,
)

# Guideline eval
guideline_eval = GuidelineEvaluator(
    guidelines=DEFAULT_GUIDELINES
    + "\nYanıt aşırı uzun olmamalıdır.\n"
    "Yanıt mümkün olduğunca özetlemeye çalışmalıdır.\n"
)  # just for example

In [22]:
typed_response = (
    response if isinstance(response, Response) else response.get_response()
)
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

Guideline eval evaluation result: The response partially answers the query but lacks specific details and statistics. It is also somewhat vague and could be more concise.
Transformed query: Here is a previous bad answer.
Sicile kayıtlı motorlu kara araçlarına ilişkin satış giderleri, aracın muhafazası, kıymet takdiri ve satış giderleri esas alınarak belirlenirken, taşınmaz mallara ilişkin satış giderleri, sadece kıymet takdiri ve satış giderleri esas alınarak belirlenir. Taşınır mallara ilişkin satış giderleri ise yine sadece kıymet takdiri ve satış giderleri esas alınarak belirlenir. Diğer mallara ilişkin satış giderleri ise niteliğine uygun düştüğü ölçüde satış gideri tablosunda yer alan ücretler esas alınır.
Here is some feedback from the evaluator about the response given.
The response partially answers the query but lacks specific details and statistics. It is also somewhat vague and could be more concise.
Now answer the question.
Please return the modified query below.


In [24]:
retry_guideline_query_engine = RetryGuidelineQueryEngine(
    base_query_engine, guideline_eval, resynthesize_query=True
)
retry_guideline_response = retry_guideline_query_engine.query(query)
print(retry_guideline_response)

The determination of sales expenses for registered motor vehicles is based on the preservation of the vehicle, valuation, and sales expenses. Similarly, for immovable properties, sales expenses are determined based on valuation and sales expenses. Sales expenses for movable properties are also determined based on valuation and sales expenses. For other goods, sales expenses are based on the fees listed in the sales expense table, according to the nature of the goods.
